In [1]:
import numpy as np
import torch
import torch_geometric
from torch_geometric.nn import VGAE, GCNConv
from torch_geometric.utils import from_networkx, erdos_renyi_graph
import networkx as nx
from scipy.stats import spearmanr

/home/maruanottoni/anaconda3/envs/gce/lib/python3.9/site-packages/torch_geometric/typing.py:72: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /home/maruanottoni/anaconda3/envs/gce/lib/python3.9/site-packages/torch_scatter/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/home/maruanottoni/anaconda3/envs/gce/lib/python3.9/site-packages/torch_geometric/typing.py:110: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /home/maruanottoni/anaconda3/envs/gce/lib/python3.9/site-packages/torch_sparse/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


# VGAE experiment

In [3]:
VGAE.__dict__

mappingproxy({'__module__': 'torch_geometric.nn.models.autoencoder',
              '__doc__': 'The Variational Graph Auto-Encoder model from the\n    `"Variational Graph Auto-Encoders" <https://arxiv.org/abs/1611.07308>`_\n    paper.\n\n    Args:\n        encoder (torch.nn.Module): The encoder module to compute :math:`\\mu`\n            and :math:`\\log\\sigma^2`.\n        decoder (torch.nn.Module, optional): The decoder module. If set to\n            :obj:`None`, will default to the\n            :class:`torch_geometric.nn.models.InnerProductDecoder`.\n            (default: :obj:`None`)\n    ',
              '__init__': <function torch_geometric.nn.models.autoencoder.VGAE.__init__(self, encoder: torch.nn.modules.module.Module, decoder: Optional[torch.nn.modules.module.Module] = None)>,
              'reparametrize': <function torch_geometric.nn.models.autoencoder.VGAE.reparametrize(self, mu: torch.Tensor, logstd: torch.Tensor) -> torch.Tensor>,
              'encode': <function torch_g

In [4]:
class Encoder(torch.nn.Module):
    def __init__(self, num_features, hidden_channels, out_channels):
        super(Encoder, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_channels)
        self.conv_mu = GCNConv(hidden_channels, out_channels)
        self.conv_logstd = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

class EarlyStopping:
    def __init__(self, patience=10, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

def generate_correlated_params(n, correlation):
    mean = [0, 0]
    cov = [[1, correlation], [correlation, 1]]
    return np.random.multivariate_normal(mean, cov, n)

def generate_graph(n, p):
    return nx.erdos_renyi_graph(n, p)

def get_embedding(model, data):
    model.eval()
    with torch.no_grad():
        z = model.encode(data.x, data.edge_index)
    return z.mean(dim=0).numpy()


# Debug train

In [ ]:
import os
import torch
from tqdm import tqdm
import numpy as np
from scipy.stats import spearmanr

def experiment(num_graphs=200, nodes=50, hidden_channels=32, out_channels=16, correlation=0.5):
    print("Starting experiment...")
    # Generate correlated parameters
    params = generate_correlated_params(num_graphs, correlation)
    # Normalize params with sigmoid
    params = torch.sigmoid(torch.tensor(params))
    print(f"Generated {num_graphs} pairs of correlated parameters")
    
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Generate graphs and prepare dataset
    all_data = []
    for i, (p1, _) in enumerate(tqdm(params, desc="Generating graphs")):
        g = generate_graph(nodes, p1)
        
        data = from_networkx(g)
        
        data.x = torch.tensor(list(dict(g.degree()).values()), dtype=torch.float).view(-1, 1).to(device)
        data.edge_index = data.edge_index.to(device)
        
        all_data.append(data)
    
    print(f"Generated {len(all_data)} graphs")

    # Create VGAE model
    model = VGAE(Encoder(1, hidden_channels, out_channels)).to(device)

    # Save data and model
    data_dir = os.path.join('..', 'data')
    os.makedirs(data_dir, exist_ok=True)
    
    torch.save(all_data, os.path.join(data_dir, 'all_data.pt'))
    torch.save(model.state_dict(), os.path.join(data_dir, 'model_initial.pt'))
    
    print(f"Saved initial data and model to {data_dir}")

    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    # Train VGAE model on the entire dataset
    print("Training VGAE model...")
    for epoch in tqdm(range(50), desc="Training epochs"):
        model.train()
        total_loss = 0
        
        for data in all_data:
            optimizer.zero_grad()
            z = model.encode(data.x, data.edge_index)
            loss = model.recon_loss(z, data.edge_index)
            loss += (1 / data.num_nodes) * model.kl_loss()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        avg_loss = total_loss / len(all_data)
        
        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1}, Avg Loss: {avg_loss:.4f}")

    print("VGAE model trained")

    # Save trained model
    torch.save(model.state_dict(), os.path.join(data_dir, 'model_trained.pt'))
    print(f"Saved trained model to {data_dir}")

    # Get embeddings for all graphs using the trained model
    embeddings = []
    
    print("Calculating embeddings...")
    model.eval()
    with torch.no_grad():
        for data in tqdm(all_data, desc="Processing graphs"):
            z = model.encode(data.x, data.edge_index)
            emb = z.mean(dim=0).cpu().numpy()
            embeddings.append(emb)

    # Save embeddings
    torch.save(torch.tensor(embeddings), os.path.join(data_dir, 'embeddings.pt'))
    print(f"Saved embeddings as tensor to {data_dir}")

    print("Experiment completed.")

print("Running experiment...")
experiment()

# Experiment

In [7]:
import os
import torch
from tqdm import tqdm
import numpy as np
from scipy.stats import spearmanr

def experiment(num_graphs=200, nodes=50, hidden_channels=32, out_channels=16, correlation=0.5):
    print("Starting experiment...")
    # Generate correlated parameters
    params = generate_correlated_params(num_graphs, correlation)
    # Normalize params with sigmoid
    params = torch.sigmoid(torch.tensor(params))
    print(f"Generated {num_graphs} pairs of correlated parameters")
    
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # Uncomment the line below if you want to use MPS (Metal Performance Shaders) on macOS
    # device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Generate graphs and prepare dataset
    all_data = []
    for i, (p1, p2) in enumerate(tqdm(params, desc="Generating graph pairs")):
        g1 = generate_graph(nodes, p1)
        g2 = generate_graph(nodes, p2)
        
        data1 = from_networkx(g1)
        data2 = from_networkx(g2)
        
        data1.x = torch.tensor(list(dict(g1.degree()).values()), dtype=torch.float).view(-1, 1).to(device)
        data2.x = torch.tensor(list(dict(g2.degree()).values()), dtype=torch.float).view(-1, 1).to(device)
        
        data1.edge_index = data1.edge_index.to(device)
        data2.edge_index = data2.edge_index.to(device)
        
        all_data.extend([data1, data2])
    
    print(f"Generated {len(all_data)} graphs")

    # Create VGAE models
    model1 = VGAE(Encoder(1, hidden_channels, out_channels)).to(device)
    model2 = VGAE(Encoder(1, hidden_channels, out_channels)).to(device)

    # Save data and models
    data_dir = os.path.join('..', 'data')
    os.makedirs(data_dir, exist_ok=True)
    
    torch.save(all_data, os.path.join(data_dir, 'all_data.pt'))
    torch.save(model1.state_dict(), os.path.join(data_dir, 'model1_initial.pt'))
    torch.save(model2.state_dict(), os.path.join(data_dir, 'model2_initial.pt'))
    
    print(f"Saved initial data and models to {data_dir}")

    optimizer1 = torch.optim.Adam(model1.parameters(), lr=0.01)
    optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.01)

    early_stopping1 = EarlyStopping(patience=5)
    early_stopping2 = EarlyStopping(patience=5)

    # Train VGAE models on the entire dataset
    print("Training VGAE models...")
    for epoch in tqdm(range(50), desc="Training epochs"):
        model1.train()
        model2.train()
        total_loss1 = 0
        total_loss2 = 0
        
        for data in all_data:
            optimizer1.zero_grad()
            z1 = model1.encode(data.x, data.edge_index)
            loss1 = model1.recon_loss(z1, data.edge_index)
            loss1 += (1 / data.num_nodes) * model1.kl_loss()
            loss1.backward()
            optimizer1.step()
            total_loss1 += loss1.item()
            
            optimizer2.zero_grad()
            z2 = model2.encode(data.x, data.edge_index)
            loss2 = model2.recon_loss(z2, data.edge_index)
            loss2 += (1 / data.num_nodes) * model2.kl_loss()
            loss2.backward()
            optimizer2.step()
            total_loss2 += loss2.item()
        
        avg_loss1 = total_loss1 / len(all_data)
        avg_loss2 = total_loss2 / len(all_data)
        
        early_stopping1(avg_loss1)
        early_stopping2(avg_loss2)
        
        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1}, Avg Loss1: {avg_loss1:.4f}, Avg Loss2: {avg_loss2:.4f}")
        
        if early_stopping1.early_stop and early_stopping2.early_stop:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break

    print("VGAE models trained")

    # Save trained models
    torch.save(model1.state_dict(), os.path.join(data_dir, 'model1_trained.pt'))
    torch.save(model2.state_dict(), os.path.join(data_dir, 'model2_trained.pt'))
    print(f"Saved trained models to {data_dir}")

    # Get embeddings for all graphs using the trained models
    embeddings1 = []
    embeddings2 = []
    
    print("Calculating embeddings...")
    for data in tqdm(all_data, desc="Processing graphs"):
        emb1 = get_embedding(model1, data)
        emb2 = get_embedding(model2, data)
        embeddings1.append(emb1)
        embeddings2.append(emb2)

    # Save embeddings
    torch.save(torch.tensor(embeddings1), os.path.join(data_dir, 'embeddings1.pt'))
    torch.save(torch.tensor(embeddings2), os.path.join(data_dir, 'embeddings2.pt'))
    print(f"Saved embeddings as tensors to {data_dir}")

    print("Calculating correlations...")
    # Calculate correlations
    param_corr, _ = spearmanr(params.cpu().numpy()[:, 0], params.cpu().numpy()[:, 1])
    emb_corr, _ = spearmanr(np.array(embeddings1).flatten(), np.array(embeddings2).flatten())

    return param_corr, emb_corr

print("Running experiment...")
param_corr, emb_corr = experiment()

Running experiment...
Starting experiment...
Generated 200 pairs of correlated parameters
Using device: cuda


Generating graph pairs: 100%|██████████| 200/200 [00:30<00:00,  6.46it/s]


Generated 400 graphs
Saved initial data and models to ../data
Training VGAE models...


Training epochs:   5%|▌         | 5/100 [01:18<24:21, 15.39s/it]

Epoch 5, Avg Loss1: 1.6474, Avg Loss2: 1.6556


...

Epoch 95, Avg Loss1: 1.6414, Avg Loss2: 1.6394


Training epochs: 100%|██████████| 100/100 [23:41<00:00, 14.22s/it]


Epoch 100, Avg Loss1: 1.6423, Avg Loss2: 1.6403
VGAE models trained
Saved trained models to ../data
Calculating embeddings...


Processing graphs:   0%|          | 0/400 [00:00<?, ?it/s]


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [9]:
def get_embedding(model, data):
    model.eval()
    with torch.no_grad():
        z = model.encode(data.x, data.edge_index)
    return z.mean(dim=0).cpu().numpy()

def load_and_process():
    print("Loading data and models...")
    data_dir = os.path.join('..', 'data')
    
    # Load data
    all_data = torch.load(os.path.join(data_dir, 'all_data.pt'))
    print(f"Loaded {len(all_data)} graphs")

    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Load models
    model1 = VGAE(Encoder(1, 32, 16)).to(device)
    model2 = VGAE(Encoder(1, 32, 16)).to(device)
    
    model1.load_state_dict(torch.load(os.path.join(data_dir, 'model1_trained.pt')))
    model2.load_state_dict(torch.load(os.path.join(data_dir, 'model2_trained.pt')))
    
    print("Models loaded successfully")

    # Calculate embeddings
    embeddings1 = []
    embeddings2 = []
    
    print("Calculating embeddings...")
    for data in tqdm(all_data, desc="Processing graphs"):
        data = data.to(device)
        emb1 = get_embedding(model1, data)
        emb2 = get_embedding(model2, data)
        embeddings1.append(emb1)
        embeddings2.append(emb2)

    # Save embeddings
    torch.save(torch.tensor(embeddings1), os.path.join(data_dir, 'embeddings1.pt'))
    torch.save(torch.tensor(embeddings2), os.path.join(data_dir, 'embeddings2.pt'))
    print(f"Saved embeddings as tensors to {data_dir}")


    print("Calculating correlations...")
    # Calculate correlations
    emb_corr, _ = spearmanr(np.array(embeddings1).flatten(), np.array(embeddings2).flatten())

    return emb_corr

print("Running processing...")
emb_corr = load_and_process()

print(f"Embedding correlation: {emb_corr}")
print("Processing completed.")

Running processing...
Loading data and models...


/tmp/ipykernel_2332/4211892069.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  all_data = torch.load(os.path.join(data_dir, 'all_data.pt'))
/tmp/ipykernel_2332/42118920

Loaded 400 graphs
Using device: cuda
Models loaded successfully
Calculating embeddings...


Processing graphs:   2%|▏         | 9/400 [00:00<00:04, 79.67it/s]

Processing graphs: 100%|██████████| 400/400 [00:04<00:00, 84.66it/s]

Saved embeddings as tensors to ../data
Calculating correlations...
Embedding correlation: 0.22352941176470587
Processing completed.


# Spectrum baseline

In [11]:
import os
import torch
import numpy as np
from tqdm import tqdm
from scipy.stats import spearmanr
import networkx as nx

def generate_correlated_params(num_graphs, correlation):
    mean = [0, 0]
    cov = [[1, correlation], [correlation, 1]]
    return np.random.multivariate_normal(mean, cov, num_graphs)

def generate_graph(nodes, p):
    return nx.erdos_renyi_graph(nodes, p)

def get_spectral_radius(graph):
    adjacency_matrix = nx.adjacency_matrix(graph).todense()
    eigenvalues = np.linalg.eigvals(adjacency_matrix)
    return np.max(np.abs(eigenvalues))

def experiment(num_graphs=200, nodes=50, correlation=0.5):
    print("Starting experiment...")
    # Generate correlated parameters
    params = generate_correlated_params(num_graphs, correlation)
    # Normalize params with sigmoid
    params = torch.sigmoid(torch.tensor(params))
    print(f"Generated {num_graphs} pairs of correlated parameters")

    # Generate graphs and calculate spectral radii
    spectral_radii1 = []
    spectral_radii2 = []

    print("Generating graphs and calculating spectral radii...")
    for p1, p2 in tqdm(params, desc="Processing graph pairs"):
        g1 = generate_graph(nodes, p1.item())
        g2 = generate_graph(nodes, p2.item())

        sr1 = get_spectral_radius(g1)
        sr2 = get_spectral_radius(g2)

        spectral_radii1.append(sr1)
        spectral_radii2.append(sr2)

    print(f"Calculated spectral radii for {num_graphs} graph pairs")

    # Save data
    data_dir = os.path.join('..', 'data')
    os.makedirs(data_dir, exist_ok=True)
    
    np.save(os.path.join(data_dir, 'params.npy'), params.numpy())
    np.save(os.path.join(data_dir, 'spectral_radii1.npy'), np.array(spectral_radii1))
    np.save(os.path.join(data_dir, 'spectral_radii2.npy'), np.array(spectral_radii2))
    
    print(f"Saved parameters and spectral radii to {data_dir}")

    print("Calculating correlations...")
    # Calculate correlations
    param_corr, _ = spearmanr(params.numpy()[:, 0], params.numpy()[:, 1])
    sr_corr, _ = spearmanr(spectral_radii1, spectral_radii2)

    return param_corr, sr_corr

print("Running experiment...")
param_corr, sr_corr = experiment()

print(f"Parameter correlation: {param_corr}")
print(f"Spectral radius correlation: {sr_corr}")

Running experiment...
Starting experiment...
Generated 200 pairs of correlated parameters
Generating graphs and calculating spectral radii...


Processing graph pairs:  48%|████▊     | 95/200 [00:01<00:01, 70.98it/s]